# Washington soil data compilation 
This notebook cleans and aggregates SSURGO data for use in Landmapper. The result is a spatial layer containing the soil attributes reported in the application. 

In [3]:
import pandas as pd
import geopandas as gpd

In [4]:
SOILS = "G:/projects/LandMapper_2020/Data/Source/Soils/2023/WA/gSSURGO_WA.gdb"

In [5]:
# read in relevant tables
# Mapunit Aggregated Attributes
muaggatt = gpd.read_file(SOILS, driver='fileGDB', layer='muaggatt')
# Component
component = gpd.read_file(SOILS, driver='fileGDB', layer='component')
# Component Forest Productivity
coforprod = gpd.read_file(SOILS, driver='fileGDB', layer='coforprod')
# Component Restrictions
corestrictions = gpd.read_file(SOILS, driver='fileGDB', layer='corestrictions')

The muaggatt table has soil attribute aggregated to the map unit level.  This is convenient, as values from this table can be reported directly. 
We will pull name, drainage class, erosion hazard, and slope.

In [6]:
# let's start with ethe muaggatt table
muaggatt_sub = muaggatt[['mukey', 'muname', 'drclassdcd', 'forpehrtdcp']]
muaggatt_sub.head(2)

,mukey,muname,drclassdcd,forpehrtdcp
0,74975,"Andic Cryochrepts, 60 to 90 percent slopes",Well drained,Severe
1,74976,"Barnhardt gravelly loam, 0 to 5 percent slopes",Well drained,Slight


In [7]:
muaggatt_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11035 entries, 0 to 11034
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mukey        11035 non-null  object
 1   muname       11035 non-null  object
 2   drclassdcd   10597 non-null  object
 3   forpehrtdcp  11035 non-null  object
dtypes: object(4)
memory usage: 345.0+ KB


The component table will just be used in relation to other tables reported at the component level (a subset of map unit).

In [8]:
# all we need from this table is the mukey - cokey crosswalk
# and the component percentage of the map unit
component_sub = component[['mukey', 'cokey', 'comppct_r']]
component_sub['comppct_p'] = component_sub['comppct_r']/100
component_sub.head(3)

C:\Users\sloreno\AppData\Local\Temp\ipykernel_38432\2825768937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  component_sub['comppct_p'] = component_sub['comppct_r']/100


,mukey,cokey,comppct_r,comppct_p
0,74975,23177119,85,0.85
1,74975,23177120,4,0.04
2,74975,23177121,3,0.03


The Component Forest Productivity table contains the site index information.

In [9]:
#first remove all null values from table
coforprod_drop = coforprod[coforprod['siteindex_r'].notna()]
coforprod_drop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 16656 entries, 2 to 41944
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   plantsym       16656 non-null  object  
 1   plantsciname   16656 non-null  object  
 2   plantcomname   16656 non-null  object  
 3   siteindexbase  16645 non-null  object  
 4   siteindex_l    702 non-null    float64 
 5   siteindex_r    16656 non-null  float64 
 6   siteindex_h    702 non-null    float64 
 7   fprod_l        0 non-null      float64 
 8   fprod_r        15345 non-null  float64 
 9   fprod_h        0 non-null      float64 
 10  cokey          16656 non-null  object  
 11  cofprodkey     16656 non-null  object  
 12  geometry       0 non-null      geometry
dtypes: float64(6), geometry(1), object(6)
memory usage: 1.8+ MB


In [10]:
# join with component table to see if there are multiple values per map unit
coforprod_key = coforprod_drop.merge(component_sub, on='cokey', how='left')
coforprod_key = coforprod_key.astype({'siteindex_r':'int'})
# if multiple values, grab first one
si_key = coforprod_key.groupby('mukey').first().reset_index()

In [11]:
si_key['si_label'] = (si_key.apply(lambda x: "{} - {} ft".format(x.plantcomname, x.siteindex_r), axis=1))
si_key = si_key[['mukey', 'si_label']]

The Component Restrictions table for depth to restrictive layer information

In [14]:
# in this table we need the cokey, 
# resdept_l - min depth to restrictive layer
# resdept_h - max depth to restrictive layer
corest_sub = corestrictions[['cokey','resdept_l', 'resdept_h', 'resdept_r']]
corest_sub.head(2)

,cokey,resdept_l,resdept_h,resdept_r
0,23177156,51.0,102.0,76.0
1,23177616,30.0,51.0,41.0


In [15]:
# merge the corest_sub and component
res = component_sub.merge(corest_sub, on='cokey', how='left')
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29996 entries, 0 to 29995
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mukey      29996 non-null  object 
 1   cokey      29996 non-null  object 
 2   comppct_r  29996 non-null  int64  
 3   comppct_p  29996 non-null  float64
 4   resdept_l  11429 non-null  float64
 5   resdept_h  11429 non-null  float64
 6   resdept_r  11552 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.6+ MB


In [16]:
# let's calcuate the percentage weighted values and sum by mukey
res['l_per'] = res['resdept_l']*res['comppct_p']
res['h_per'] = res['resdept_h']*res['comppct_p']

res.head(10)

,mukey,cokey,comppct_r,comppct_p,resdept_l,resdept_h,resdept_r,l_per,h_per
0,74975,23177119,85,0.85,NaN,NaN,NaN,NaN,NaN
1,74975,23177120,4,0.04,NaN,NaN,NaN,NaN,NaN
2,74975,23177121,3,0.03,NaN,NaN,NaN,NaN,NaN
3,74975,23177122,2,0.02,NaN,NaN,NaN,NaN,NaN
4,74975,23177123,3,0.03,NaN,NaN,NaN,NaN,NaN
5,74975,23177124,3,0.03,NaN,NaN,NaN,NaN,NaN
6,74976,23177155,95,0.95,NaN,NaN,NaN,NaN,NaN
7,74976,23177156,3,0.03,51.0,102.0,76.0,1.53,3.06
8,74976,23177157,2,0.02,NaN,NaN,NaN,NaN,NaN
9,74977,23177613,90,0.90,NaN,NaN,NaN,NaN,NaN


In [18]:
res_comp = res.groupby('mukey').agg({'l_per':sum, 'h_per':sum}).reset_index()

## Let's join our tables together

In [20]:
join = muaggatt_sub.merge(si_key, on='mukey', how='left')
join = join.merge(res_comp, on='mukey', how='left')
join['drclassdcd'].fillna("No Data Available", inplace=True)
join['si_label'].fillna('None', inplace=True)
join.rename(columns={'l_per': 'avg_rs_l', 'h_per': 'avg_rs_h'}, inplace=True)
join.head(10)

,mukey,muname,drclassdcd,forpehrtdcp,si_label,avg_rs_l,avg_rs_h
0,74975,"Andic Cryochrepts, 60 to 90 percent slopes",Well drained,Severe,western hemlock - 89 ft,0.00,0.00
1,74976,"Barnhardt gravelly loam, 0 to 5 percent slopes",Well drained,Slight,Douglas-fir - 106 ft,1.53,3.06
2,74977,"Lynden sandy loam, 3 to 8 percent slopes",Well drained,Moderate,Douglas-fir - 112 ft,0.90,1.53
3,74978,"Lynden-Urban land complex, 0 to 3 percent slopes",Well drained,Slight,Douglas-fir - 112 ft,0.90,1.53
4,74979,"Lynnwood sandy loam, 0 to 5 percent slopes",Somewhat excessively drained,Moderate,Douglas-fir - 121 ft,0.60,1.02
5,74980,"Lynnwood sandy loam, 5 to 20 percent slopes",Somewhat excessively drained,Severe,Douglas-fir - 121 ft,0.30,0.51
6,74981,"Montborne gravelly loam, 5 to 30 percent slopes",Moderately well drained,Severe,Douglas-fir - 114 ft,43.35,86.70
7,74982,"Montborne gravelly loam, 30 to 60 percent slopes",Moderately well drained,Severe,Douglas-fir - 114 ft,43.35,86.70
8,74983,"Montborne-Rinker complex, 30 to 60 percent slopes",Moderately well drained,Severe,Douglas-fir - 107 ft,43.35,86.70
9,74984,"Mt. Vernon fine sandy loam, 0 to 2 percent slopes",Moderately well drained,Slight,Douglas-fir - 130 ft,0.00,0.00


## Bring in the map units shapefile and join with attributes

In [21]:
washington = gpd.read_file(SOILS, driver="FileGDB", layer= 'MUPOlYGON')
washington = washington.to_crs(3857)
washington.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 549880 entries, 0 to 549879
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   AREASYMBOL    549880 non-null  object  
 1   SPATIALVER    549880 non-null  float64 
 2   MUSYM         549880 non-null  object  
 3   MUKEY         549880 non-null  object  
 4   Shape_Length  549880 non-null  float64 
 5   Shape_Area    549880 non-null  float64 
 6   geometry      549880 non-null  geometry
dtypes: float64(3), geometry(1), object(3)
memory usage: 29.4+ MB


In [22]:
export = washington.merge(join, left_on="MUKEY", right_on='mukey', how='left')
export.drop('mukey', axis=1, inplace = True)
export.insert(0, 'id', range(0, 0 + len(export)))
export.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 549880 entries, 0 to 549879
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            549880 non-null  int64   
 1   AREASYMBOL    549880 non-null  object  
 2   SPATIALVER    549880 non-null  float64 
 3   MUSYM         549880 non-null  object  
 4   MUKEY         549880 non-null  object  
 5   Shape_Length  549880 non-null  float64 
 6   Shape_Area    549880 non-null  float64 
 7   geometry      549880 non-null  geometry
 8   muname        549880 non-null  object  
 9   drclassdcd    549880 non-null  object  
 10  forpehrtdcp   549880 non-null  object  
 11  si_label      549880 non-null  object  
 12  avg_rs_l      549880 non-null  float64 
 13  avg_rs_h      549880 non-null  float64 
dtypes: float64(5), geometry(1), int64(1), object(7)
memory usage: 58.7+ MB


In [23]:
export.to_file('C:/Users/sloreno/LandMapper/data/Soil_Attributes/WA_soils.shp', driver='ESRI Shapefile')

C:\Users\sloreno\AppData\Local\Temp\ipykernel_38432\90305232.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  export.to_file('C:/Users/sloreno/LandMapper/data/Soil_Attributes/WA_soils.shp', driver='ESRI Shapefile')
